In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors, preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
data = pd.read_csv('AthleteResults.csv')

In [3]:
data.head()

,First_Sex,First_Equipment,First_BodyweightKg,First_Wilks,First_WhereHeld,First_EventDate,First_Competition_experience,Second_Sex,Second_Equipment,Second_BodyweightKg,...,Second_EventDate,Second_Competition_experience,Third_Sex,Third_Equipment,Third_BodyweightKg,Third_Wilks,Third_WhereHeld,Third_EventDate,Third_Competition_experience,birth_year
0,1,3,100.0,418.40,SoPac,2018-01-27,1,1,1,87.20,...,2017-09-30,2,1,3,100.00,413.84,SoPac,2017-09-09,3,1992.0
1,1,3,99.8,493.35,SoPac,2018-01-27,1,1,3,102.60,...,2017-09-09,2,1,3,99.50,500.05,SoPac,2017-07-21,3,1989.0
2,1,3,98.8,334.84,SoPac,2018-01-27,1,1,1,99.30,...,2017-10-07,2,1,3,93.80,340.99,SoPac,2012-11-09,3,1968.0
3,1,3,98.0,395.80,SoPac,2018-01-27,1,1,3,89.80,...,2016-07-08,2,1,3,89.80,399.45,SoPac,2014-11-21,3,1969.0
4,1,3,89.7,380.49,SoPac,2018-01-27,1,1,2,74.39,...,2017-02-23,2,1,2,74.57,306.66,NAmerica,2017-02-18,3,1990.0


In [4]:
# data['1st_Sex']
data.query('First_Sex == 0').head()

,First_Sex,First_Equipment,First_BodyweightKg,First_Wilks,First_WhereHeld,First_EventDate,First_Competition_experience,Second_Sex,Second_Equipment,Second_BodyweightKg,...,Second_EventDate,Second_Competition_experience,Third_Sex,Third_Equipment,Third_BodyweightKg,Third_Wilks,Third_WhereHeld,Third_EventDate,Third_Competition_experience,birth_year
14,0,3,54.61,312.99,NAmerica,2018-01-27,1,0,3,55.88,...,2017-01-28,2,0,3,36.20,185.18,NAmerica,2012-06-23,3,2003.0
15,0,3,57.79,320.23,NAmerica,2018-01-27,1,0,3,58.88,...,2017-11-18,2,0,3,58.97,261.40,NAmerica,2017-11-04,3,2004.0
16,0,1,85.18,335.44,NAmerica,2018-01-27,1,0,1,88.00,...,2017-11-04,2,0,1,89.72,298.26,NAmerica,2017-10-07,3,1996.0
17,0,3,88.60,315.31,SoPac,2018-01-27,1,0,3,98.30,...,2017-09-09,2,0,3,99.00,288.10,SoPac,2016-09-03,3,1985.0
21,0,3,77.40,396.34,SoPac,2018-01-27,1,0,3,80.70,...,2017-07-21,2,0,3,73.90,405.42,SoPac,2017-03-18,3,1989.0


In [5]:
data.dtypes

First_Sex                          int64
First_Equipment                    int64
First_BodyweightKg               float64
First_Wilks                      float64
First_WhereHeld                   object
First_EventDate                   object
First_Competition_experience       int64
Second_Sex                         int64
Second_Equipment                   int64
Second_BodyweightKg              float64
Second_Wilks                     float64
Second_WhereHeld                  object
Second_EventDate                  object
Second_Competition_experience      int64
Third_Sex                          int64
Third_Equipment                    int64
Third_BodyweightKg               float64
Third_Wilks                      float64
Third_WhereHeld                   object
Third_EventDate                   object
Third_Competition_experience       int64
birth_year                       float64
dtype: object

In [6]:
data['First_WhereHeld'] = LabelEncoder().fit_transform(data['First_WhereHeld'])
data['Second_WhereHeld'] = LabelEncoder().fit_transform(data['Second_WhereHeld'])
data['Third_WhereHeld'] = LabelEncoder().fit_transform(data['Third_WhereHeld'])

In [7]:
#Ask about this

In [8]:
data.drop('Third_Sex', axis=1, inplace=True)
data.drop('Second_Sex', axis=1, inplace=True)

In [9]:
data['First_EventDate'] = pd.to_datetime(data['First_EventDate'])
data['Second_EventDate'] = pd.to_datetime(data['Second_EventDate'])
data['Third_EventDate'] = pd.to_datetime(data['Third_EventDate'])


data['First_Rest'] = data['First_EventDate'] - data['Second_EventDate']
data['Second_Rest'] = data['Second_EventDate'] - data['Third_EventDate']
#not_features = ['First_EventDate', 'Second_EventDate', 'Third_EventDate', 'First_Wilks']

In [10]:
data['First_Rest']=data['First_Rest'].dt.days
data['Second_Rest'] = data['Second_Rest'].dt.days

In [11]:
features = data.columns
not_features = ['First_EventDate', 'Second_EventDate', 'Third_EventDate', 'First_Wilks']
features = features.drop(not_features)
y = data.First_Wilks
X = data[features]

In [12]:
#X

In [13]:
y.dtype

dtype('float64')

In [14]:
data.head()

,First_Sex,First_Equipment,First_BodyweightKg,First_Wilks,First_WhereHeld,First_EventDate,First_Competition_experience,Second_Equipment,Second_BodyweightKg,Second_Wilks,...,Second_Competition_experience,Third_Equipment,Third_BodyweightKg,Third_Wilks,Third_WhereHeld,Third_EventDate,Third_Competition_experience,birth_year,First_Rest,Second_Rest
0,1,3,100.0,418.40,5,2018-01-27,1,1,87.20,355.37,...,2,3,100.00,413.84,5,2017-09-09,3,1992.0,119,21
1,1,3,99.8,493.35,5,2018-01-27,1,3,102.60,485.08,...,2,3,99.50,500.05,5,2017-07-21,3,1989.0,140,50
2,1,3,98.8,334.84,5,2018-01-27,1,1,99.30,329.57,...,2,3,93.80,340.99,5,2012-11-09,3,1968.0,112,1793
3,1,3,98.0,395.80,5,2018-01-27,1,3,89.80,410.63,...,2,3,89.80,399.45,5,2014-11-21,3,1969.0,568,595
4,1,3,89.7,380.49,5,2018-01-27,1,2,74.39,331.56,...,2,2,74.57,306.66,3,2017-02-18,3,1990.0,338,5


In [15]:
print(X.shape, y.shape)

(12254, 18) (12254,)


# Linear Regression using various methods

AKA: my simple starting point

In [16]:
from scipy import linspace, polyval, polyfit, sqrt, stats, randn, optimize
import statsmodels.api as sm
import time

In [17]:
print(X.shape, y.shape)

(12254, 18) (12254,)


In [18]:
np.asarray(data)

array([[1, 3, 100.0, ..., 1992.0, 119, 21],
       [1, 3, 99.8, ..., 1989.0, 140, 50],
       [1, 3, 98.8, ..., 1968.0, 112, 1793],
       ..., 
       [1, 1, 103.0, ..., 1993.0, 197, 327],
       [1, 1, 85.4, ..., 1996.0, 147, 76],
       [1, 1, 92.5, ..., 1998.0, 181, 170]], dtype=object)

In [19]:
X.dtypes

First_Sex                          int64
First_Equipment                    int64
First_BodyweightKg               float64
First_WhereHeld                    int64
First_Competition_experience       int64
Second_Equipment                   int64
Second_BodyweightKg              float64
Second_Wilks                     float64
Second_WhereHeld                   int64
Second_Competition_experience      int64
Third_Equipment                    int64
Third_BodyweightKg               float64
Third_Wilks                      float64
Third_WhereHeld                    int64
Third_Competition_experience       int64
birth_year                       float64
First_Rest                         int64
Second_Rest                        int64
dtype: object

In [20]:
model = sm.OLS(y, X).fit()
sm.add_constant(X)
predictions = model.predict(X)

In [21]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            First_Wilks   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     3324.
Date:                Fri, 06 Apr 2018   Prob (F-statistic):               0.00
Time:                        21:16:53   Log-Likelihood:                -60374.
No. Observations:               12254   AIC:                         1.208e+05
Df Residuals:                   12238   BIC:                         1.209e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
First_Sex     

In [22]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=167)
#scaler = preprocessing.StandardScaler().fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)




In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_test_pred = lr.predict(X_test)
lr_train_pred=lr.predict(X_train)
lr.score(X_test, y_test) #Rsq score of our model

0.81360485846482944

In [24]:
#svr = SVC(kernel='linear')
gnb = GaussianNB #for classification later
knn = neighbors.KNeighborsRegressor(n_neighbors=5, weights='distance', algorithm='auto')


model_knn = knn.fit(X_train, y_train)
#svr.fit(X_train, y_train)
#model_gnb = gnb.fit(X_train, y_train)

In [25]:
knn_pred = knn.predict(X_test)
knn_actual= y_test
mse = (((knn_pred - knn_actual)**2).sum())/len(knn_pred)
#accuracy_score(y_test, knn_pred)
print(mse)

1278.88184025


In [26]:
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)

In [ ]:
y_rbf = svr_rbf.fit(X, y).predict(X)

In [ ]:
lin_model= svr_lin.fit(X, y)

In [ ]:
y_lin = lin_model.predict(X)

In [ ]:
y_poly = svr_poly.fit(X, y).predict(X)

In [ ]:
#Support Vector Results
lw = 2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_rbf, color='navy', lw=lw, label='RBF model')
plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
plt.plot(X, y_poly, color='cornflowerblue', lw=lw, label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

In [ ]:
# First Run Neural Network
X.shape

In [ ]:
#Initialize Neural Network


In [ ]:
def baseline_model():
    model = Sequential()
    #Adding Input layer and hidden layer
    model.add(Dense(6, input_dim=18, kernel_initializer='normal', activation='relu'))

#Add second layer
    model.add(Dense(1, kernel_initializer='normal'))

#Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
#Establish method to compare model accuracy
seed = 6
np.random.seed(seed)

#Evaluate model
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=128, verbose=1)

In [ ]:
#evaluate model
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
#print('Results:', results.mean(), results.std())
#results

In [ ]:
#Baseline performance

In [ ]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=100, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
#print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

<<<<<<<<<<<------------This is clearly not working well...

In [ ]:
def larger_model():
    model=Sequential()
    model.add(Dense(28, input_dim=18, kernel_initializer='normal', activation='relu'))
    model.add(Dense(18, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=100, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
print(X.shape, X_test.shape, X_train.shape)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# define model
def new_baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(1, input_dim=18, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
regr = new_baseline_model()
regr.fit(X_train, y_train,          
          nb_epoch=100, validation_split=0.2, verbose = 1) #batch_size=5,

In [ ]:
knn_pred

In [ ]:
lr_test_pred

In [ ]:
lr_train_pred